In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math

iris_df = pd.read_csv("spambase.csv")

real_iris_df = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
real_iris_df = real_iris_df.replace({'Iris-setosa':0, 'Iris-versicolor': 1, 'Iris-virginica': 2})

In [2]:
# spam = pd.read_csv("spambase.csv")
# len(spam.iloc[:,1])
# len(np.unique(spam.iloc[:,1]))


# HELPER FUNCTIONS

## informational gain

In [3]:
def info_gain(dfp, df1, df2):
    
    """ returns an informational gain of a "class" column of given dataframse
    
    type dfp, df1, df2: pandas dataframes
    
    rtype: float
    
    """
    def entropy(y):
        """ returns the entropy of values in the y
    
        type y: pandas df consisting of one column
        rtype: float

        """
        if y.empty:
            return 0

        els, counts = np.unique(y, return_counts=True)
        
        # if the set if homogeneous 
        if len(els) == 1:
            return 0  # Entropy is zero if there's no variance
        
        probs = counts / len(y)
        return -np.sum(probs * np.log2(probs))
    
    n, m1, m2 = len(dfp), len(df1), len(df2)
    
    if n == 0 or m1 == 0 or m2 == 0:
        return 0
    
    internal_entropy = (entropy(df1.iloc[:,-1]) * m1 / n) + (entropy(df2.iloc[:,-1]) * m2 / n)
    parent_entropy = entropy(dfp.iloc[:,-1])
    
    return parent_entropy - internal_entropy

In [4]:
# testing
# y = [1,2,2,2,2,3,3,3]
# els, counts = np.unique(y, return_counts=True)
# print(els, counts)
# probs = counts/len(y)
# probs
# np.sum(np.log2(probs)*probs)

## splitting the dataset

In [4]:
def best_threshold(column, df):
    """ iterates through all values in  column of df, 
    finds the best column, that splits with the most info gain
    
    type column: one column df 
    type df: a dataframe 
    
    returns:
    
    best split value, corresponding info gain
    rtype: (float, float) 
    
    """
    best_split = (0,0)
    
    for i, val in enumerate(column):
        if i != len(column)-1:
            avg = (val + column.iloc[i+1])/2
            class1, class2 = df[column>avg], df[column<=avg]
            informational_gain = info_gain(df, class1, class2) 

            if best_split[1] < informational_gain:
                best_split = (avg, informational_gain)

    return best_split

def best_column(df):
    """Iterates through all columns of df except the last one, 
    calculates the best_threshold of each column.
    
    Returns: 
    (threshold, info_gain, col_name) of the column with the best info gain.
    """

    best_split = (0, 0, "")
    
    # Iterate through all columns except the last one
    for col_name in df.columns[:-1]:
        threshold, info_gain = best_threshold(df[col_name], df)
        if best_split[1] < info_gain:
            best_split = threshold, info_gain, col_name
            
    return best_split  # (threshold, info_gain, col_name)

# print(best_column(iris_df))

def split_df(df):
    
    threshold, info_gain, best_col = best_column(df)
    
    left_df = df[df[best_col]>threshold]
    right_df = df[df[best_col]<=threshold]
    
    return threshold, info_gain, best_col, left_df, right_df

In [6]:
#testing
# print("best threshold for the petal_width feature \n", best_threshold(iris_df["petal_width"], iris_df),"\n")
# print("best threshold for the petal_length feature \n", best_threshold(iris_df["petal_length"], iris_df),"\n")
# print("best threshold for the sepal_length feature \n",best_threshold(iris_df["sepal_length"], iris_df),"\n")
# print("best threshold for the sepal_width feature \n",best_threshold(iris_df["sepal_width"], iris_df),"\n")
# print("\n Best first split of the dataset: \n")
# print(split_df(iris_df))

# DECISION TREE BUILDING

## node class

In [5]:
class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, depth=None, info_gain=None, leaf=None, df = None):
        self.feature = feature 
        self.threshold = threshold
        self.left = left
        self.right = right
        self.leaf = leaf
        self.depth = depth
        self.info_gain = info_gain
        self.df = df 

    def __str__(self):
        node_representation = ""
        if self.leaf is not None:
            node_representation = f"Node Depth: {self.depth}, Leaf: {self.leaf}, Df: {self.df}"
        else:
            left_child = "Present" if self.left else "None"
            right_child = "Present" if self.right else "None"
            node_representation = (f"Node Depth: {self.depth}\n"
                                   f"Feature: {self.feature}\n"
                                   f"Threshold: {self.threshold}\n"
                                   f"Information Gain: {self.info_gain}\n"
                                   f"Dataset used: {self.df}\n")

        return node_representation + "\n" 

def build_tree(df, min_sample_size, depth=0, max_depth=100):
    if len(df) <= min_sample_size or depth >= max_depth or len(np.unique(df.iloc[:,-1])) == 1:
        
        leaf_value = df.iloc[:,-1].mode()[0]
        return DecisionTreeNode(depth = depth, leaf=leaf_value, df=df)

    threshold, info_gain, best_col, df_left, df_right = split_df(df)
    
#     print(df_left, "\n length:", len(df_left))
#     print(df_right, "\n length:", len(df_right))
    
    left_node = build_tree(df_left, min_sample_size, depth + 1)
    right_node = build_tree(df_right,min_sample_size, depth + 1)

    return DecisionTreeNode(feature=best_col, threshold=threshold,left=left_node, right=right_node, depth = depth, info_gain = info_gain,leaf = None, df=df)

# TESTING

## predict and find accuracy 

In [6]:
# train_df, test_df = train_test_split(iris_df, test_size=0.2, random_state=15) 
# # tree = build_tree(df=train_df, min_sample_size =23)

def predict_all(tree_root, test_df):
    def traverse(tree_node, x):
        if tree_node.leaf is not None:
            return tree_node.leaf

        threshold, feature = tree_node.threshold, tree_node.feature
        
        if x[feature] > threshold:
            return traverse(tree_node.left, x)
        else:
            return traverse(tree_node.right, x)
    
    
    predicted_list = []
    for _, x in test_df.iterrows():
        pred_x = traverse(tree_root, x)
        predicted_list.append(pred_x)
        
    return predicted_list

# print(predict_all(tree, test_df))
# print(test_df.iloc[:,-1])
# print(accuracy_score(test_df.iloc[:,-1],predict_all(tree, test_df)))

### accuracy and accuracy scores for a particular n_min

In [9]:
from sklearn.model_selection import KFold

def cross_validate(data, k=10):
    kf = KFold(n_splits=k, shuffle=True, random_state=15)
    
    accuracy_scores = []

    for train_index, test_index in kf.split(data):
        train_df, test_df = data.iloc[train_index], data.iloc[test_index]
        
        tree = build_tree(df=train_df, min_sample_size=5)
        predictions = predict_all(tree, test_df)
        
        accuracy = accuracy_score(test_df.iloc[:,-1], predictions)
        accuracy_scores.append(accuracy)

    return k, accuracy_scores, np.mean(accuracy_scores), np.std(accuracy_scores)

k, accuracy_scores, mean_accuracy, std_dev = cross_validate(iris_df)
print(f"Accuracy scores for {k}th fold are {accuracy_scores}")
print("Average Accuracy:", mean_accuracy)
print("Standard Deviation:", std_dev)


Accuracy scores for 10th fold are [0.9282608695652174, 0.9347826086956522, 0.9282608695652174, 0.9152173913043479, 0.9043478260869565, 0.9217391304347826, 0.9195652173913044, 0.9478260869565217, 0.9326086956521739, 0.9152173913043479]
Average Accuracy: 0.9247826086956522
Standard Deviation: 0.011633989704573593


## predict for different n_min for spambase


In [10]:
from sklearn.model_selection import KFold

def cross_validate_n_min(df, n_min, k=10):
    kf = KFold(n_splits=k, shuffle=True, random_state=10)
    
    accuracy_scores = []

    for train_index, test_index in kf.split(df):
        train_df, test_df = df.iloc[train_index], df.iloc[test_index]
        
        tree = build_tree(df = train_df, min_sample_size = n_min)
        predictions = predict_all(tree, test_df)
        
        accuracy = accuracy_score(test_df.iloc[:,-1], predictions)
        accuracy_scores.append(accuracy)

    return k, accuracy_scores, np.mean(accuracy_scores), np.std(accuracy_scores)

k, accuracy_scores, mean_accuracy, std_dev = cross_validate_n_min(iris_df,n_min=10)
print(f"Accuracy scores for {k}th fold are {accuracy_scores}")
print("Average Accuracy:", mean_accuracy)
print("Standard Deviation:", std_dev)

Accuracy scores for 10th fold are [0.9478260869565217, 0.9108695652173913, 0.9326086956521739, 0.9347826086956522, 0.9326086956521739, 0.9260869565217391, 0.9347826086956522, 0.9478260869565217, 0.9326086956521739, 0.908695652173913]
Average Accuracy: 0.9308695652173913
Standard Deviation: 0.012366489263763387


In [11]:
n_mins = [230,460,690,920]
summary = []

for n_min in n_mins:
    k, accuracy_scores, mean_accuracy, std_dev = cross_validate_n_min(df=iris_df, n_min=n_min)
    summary.append({
        'n_min': n_min,
        'mean_accuracy': mean_accuracy,
        'std_dev': std_dev,
        'accuracy_scores': accuracy_scores
    })

summary_df = pd.DataFrame(summary)
summary_df.set_index('n_min', inplace=True)
print(summary_df)

       mean_accuracy   std_dev  \
n_min                            
230         0.903261  0.009637   
460         0.890870  0.017359   
690         0.880000  0.024591   
920         0.838913  0.022685   

                                         accuracy_scores  
n_min                                                     
230    [0.9130434782608695, 0.8891304347826087, 0.891...  
460    [0.8978260869565218, 0.8717391304347826, 0.860...  
690    [0.8891304347826087, 0.8717391304347826, 0.852...  
920    [0.8282608695652174, 0.8717391304347826, 0.852...  


## predict for different n_min for iris dataset


In [7]:
from sklearn.model_selection import KFold

def cross_validate_n_min(df, n_min, k=10):
    kf = KFold(n_splits=k, shuffle=True, random_state=10)
    
    accuracy_scores = []

    for train_index, test_index in kf.split(df):
        train_df, test_df = df.iloc[train_index], df.iloc[test_index]
        
        tree = build_tree(df = train_df, min_sample_size = n_min)
        predictions = predict_all(tree, test_df)
        
        accuracy = accuracy_score(test_df.iloc[:,-1], predictions)
        accuracy_scores.append(accuracy)

    return k, accuracy_scores, np.mean(accuracy_scores), np.std(accuracy_scores)

# k, accuracy_scores, mean_accuracy, std_dev = cross_validate_n_min(iris_df,n_min=10)
# print(f"Accuracy scores for {k}th fold are {accuracy_scores}")
# print("Average Accuracy:", mean_accuracy)
# print("Standard Deviation:", std_dev)

n_mins = [5,10,15,20]

summary = []

for n_min in n_mins:
    k, accuracy_scores, mean_accuracy, std_dev = cross_validate_n_min(df=real_iris_df, n_min=n_min)
    summary.append({
        'n_min': n_min,
        'mean_accuracy': mean_accuracy,
        'std_dev': std_dev,
        'accuracy_scores': accuracy_scores
    })

summary_df = pd.DataFrame(summary)
summary_df.set_index('n_min', inplace=True)
print(summary_df)

       mean_accuracy   std_dev  \
n_min                            
5           0.960000  0.061101   
10          0.966667  0.061464   
15          0.966667  0.061464   
20          0.966667  0.061464   

                                         accuracy_scores  
n_min                                                     
5      [1.0, 1.0, 1.0, 0.9333333333333333, 0.8, 1.0, ...  
10     [1.0, 1.0, 1.0, 0.9333333333333333, 0.8, 1.0, ...  
15     [1.0, 1.0, 1.0, 0.9333333333333333, 0.8, 1.0, ...  
20     [1.0, 1.0, 1.0, 0.9333333333333333, 0.8, 1.0, ...  
